# DART 보험사 재무제표 자동 다운로드 및 데이터 통합 프로그램 개발

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
import pandas as pd
import time
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import OpenDartReader
import os
import seaborn as sns
import sys

In [2]:
api_key = '	31718c7bf232574ee78e6f3f81c922043baad322' # 본인의 api key 입력
dart = OpenDartReader(api_key)

sys.maxsize
set_matplotlib_formats('retina')
pd.options.display.max_rows = 50
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus = False)

pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = 60
pd.options.display.max_rows = 100

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

#  opendart 이용, 보고서 이름과 rcept_no, 회사 이름 리스트 형식 변환

In [3]:
corp_code = {'삼성생명':'032830',
            '한화생명':'088350',
            '미래에셋생명':'085620'}

# 분석하고자 하는 회사 명

dart_list = dart.list(corp_code['한화생명'], start='1999-01-01', kind='A') 


# 몇 기간의 데이터를 가져올 건지

data_size = 6



# 보고서 기간 타입 결정

report_type_dict = {
    '사업보고서':'A001',
    '반기보고서':'A002',
    '분기보고서':'A003',
}

report_type_selected = list(report_type_dict.keys())[1]

dart_list = dart_list[['corp_name','report_nm','rcept_no']]
dart_list_selected = dart_list[dart_list['report_nm'].str.contains(report_type_selected)].copy()

In [4]:
corp_nms = list(dart_list_selected['corp_name'])
corp_nm = corp_nms[0] # 회사 이름 ex) 삼성생명

report_nms = list(dart_list_selected['report_nm'])
report_nm = report_nms[0] # 리포트 이름 ex) 반기보고서 (2020.09)

# 분석하려는 보고서 유형 선택

search_df_nms = ['연결 포괄손익계산서','연결 재무상태표','현금흐름표','자본변동표']
search_df_nm = f'{corp_nm} {search_df_nms[0]}'

rcept_nos = list(dart_list_selected['rcept_no'])

In [5]:
date_list = list(dart_list_selected['report_nm'].str.split(' ').str[-1])

date_list_mod=[]

for i, val in enumerate(date_list):
    date = date_list[i].replace('(','').replace(')','')
    date_list_mod.append(date)

In [6]:
df_date = pd.Series(date_list_mod)

# url 딕셔너리 설정

In [7]:
urls = []

for rcpNo in rcept_nos:
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcpNo)

    params = {
        'crtfc_key' : '31718c7bf232574ee78e6f3f81c922043baad322',
        'corp_code' : '032830',
        'bgn_de' : '19990101',
        'pblntf_detail_ty': '{}'.format(report_type_dict[report_type_selected]),
        'page_count': '100',
    }

    res = requests.get(url,params=params, headers = headers)
    res.raise_for_status()
    
    urls.append(res.url)

# 제무제표 데이터 추출

## 재무제표 파일 다운로드하는 data_to_csv 함수 만들기

### selenium 활용 파일 다운로드 함수

In [8]:
def rawdata_to_csv(urls):
    for i, url in enumerate(urls):
        browser = webdriver.Chrome(options = options)
        browser.maximize_window()
        browser.get(url)
        browser.find_element_by_xpath('//*[@id="ext-gen10"]/div/li[5]/ul/li[2]/div/a/span').click()
        iframes = browser.find_elements_by_tag_name('iframe')
        browser.switch_to.frame(iframes[0])
        html = browser.page_source

        report_nm = report_nms[i]
        tables = pd.read_html(html)

        df_list= list()

        for idx, v in enumerate(tables):
            if idx in [0,2,4,6]:
                pass
            else:
                df = tables[idx]
                df_list.append(df)

            for index, value in enumerate(df_list):
                if index == 0:
                    df_nm = '연결 재무상태표'
                elif index == 1:
                    df_nm = '연결 포괄손익계산서'
                elif index == 2:
                    df_nm = '자본변동표'
                else:
                    df_nm = '현금흐름표'

                df_list[index].to_csv(f'{corp_nm} {df_nm} {report_nm}.csv', encoding='utf-8-sig')
    browser.quit()

### url 사이즈 선택

In [9]:
urls = urls[:data_size]
rawdata_to_csv(urls)

# 재무제표 데이터 수정, 병합, 통합 파일 전처리 및 저장

## merge 위해 데이터 수정

In [10]:
data_list = os.listdir()

match_list = list()

for df_nm in data_list:
    if search_df_nm in df_nm:
        match_list.append(df_nm)
        
match_list      

['한화생명 연결 포괄손익계산서 반기보고서 (2015.06).csv',
 '한화생명 연결 포괄손익계산서 반기보고서 (2016.06).csv',
 '한화생명 연결 포괄손익계산서 반기보고서 (2017.06).csv',
 '한화생명 연결 포괄손익계산서 반기보고서 (2018.06).csv',
 '한화생명 연결 포괄손익계산서 반기보고서 (2019.06).csv',
 '한화생명 연결 포괄손익계산서 반기보고서 (2020.06).csv']

## 데이터 전처리 후 통합 및 파일 저장

### 다운로드 받은 raw data read후 전처리

In [11]:
# 한화생명 단위 : 원
# 삼성생명 단위 : 백만 원 , 단위 모두 백만 원으로 처리하자.
# 윗 줄 기존 코드 open 에 백업함

df_list = list()

for i, df_nm in enumerate(match_list):
    df = pd.read_csv('./{}'.format(df_nm), encoding='utf-8-sig')
    if corp_nm == '삼성생명':
        df = df.iloc[:,1:3]
        df = df[df['과 목'].notnull()]
    else:
        df = df.drop(0) # 타보험사 수정
        df = df.iloc[:,1:5].drop('주 석', axis=1)
        df = df.iloc[:,::2]

    df['과 목'] = df['과 목'].str.split('.').str[-1]
    df['과 목'] = df['과 목'].str.split('(').str[0]
    df['과 목'] = df['과 목'].str.strip().copy()
    df.rename(columns = {'과 목' : '과목'}, inplace = True)
    df_list.append(df)

### colunm to index

In [12]:
df_list_index = list()

for i, val in enumerate(df_list):
    df_index = df_list[i].set_index('과목').copy()
    df_list_index.append(df_index)

### join 으로 data 통합 ( 각 보고 시점에만 존재하는 계정들은 제외, merge 에서 left, right_index = True & how = 'inner'와 같음)

In [13]:
for i, val in enumerate(df_list_index):
    if i == 0:
        df_join = df_list_index[i]
    elif i <= len(df_list_index):
        df_join = df_join.join(df_list_index[i])
    else:
        continue

## 통합 파일 전처리

### 중복행 제거 및 칼럼 날짜 데이터로 변경

In [14]:
df_date_cols = df_date.loc[:len(match_list)-1].sort_index(ascending=False)

In [15]:
date_cols = pd.to_datetime(df_date_cols)
df_join = df_join.reset_index().copy()
df_join_dup = df_join.drop_duplicates('과목', keep="first").copy()
df_join_dup = df_join_dup.set_index(['과목']).copy()
df_join_dup.columns = df_date_cols

### 데이터프레임 transpose 및 내부 데이터 int 형으로 변경

In [16]:
def str_to_int(x):
    if isinstance(x, str):
        x = x.replace('-','0')
        x = x.replace('(','-')
        x = x.replace(')','')
        x = x.replace(',','')
        x = int(x)
    return x

In [17]:
df_join_t = df_join_dup.T.fillna(0)
for i in range(len(df_join_t.index)):
    df_join_t.iloc[i] = df_join_t.iloc[i].apply(str_to_int)

### 데이터 단위 백만 원으로 변경

In [18]:
if corp_nm == '한화생명':
    df_join_t = df_join_t / 1000000

In [19]:
df_join_t = df_join_t.astype(int).copy()

In [20]:
df_join_t = df_join_t.reset_index()

In [21]:
df_join_t.insert(0, '회사명', corp_nm)

In [22]:
df_join_t

과목,회사명,index,<후속적으로 당기손익으로 재분류되지 않는 항목>,<후속적으로 당기손익으로 재분류될 수 있는 항목>,기타비용,기타수익,기타포괄손익,대손상각비,매도가능금융자산평가손익,반기순이익,반기순이익의 귀속,반기총포괄손익,반기총포괄손익의 귀속,법인세비용,법인세비용차감전순이익,보통주 기본 및 희석주당순이익,보험계약부채전입액,보험료수익,비지배지분,사업비,신계약비상각비,영업비용,영업수익,영업외비용,영업외수익,영업이익,외환거래 및 환산손실,외환거래 및 환산이익,유가증권평가 및 처분손실,유가증권평가 및 처분이익,유형자산재평가이익,이자수익,재보험비용,재보험수익,재산관리비,주당이익,지급보험금,지배기업의 소유주,지분법자본변동,특별계정기타포괄손익,특별계정비용,특별계정수익,특별계정수입수수료,특별계정지급수수료,파생상품평가 및 거래손실,파생상품평가 및 거래이익,해외사업장환산외환차이,현금흐름위험회피 파생상품평가손익,NaN
0,한화생명,2015.06,0,0,169545,285426,-1883,13894,-45506,421220,421220,419336,419336,123247,544468,0,2521364,4780235,-52,421277,604445,7490439,8013635,23913,45186,523195,212938,430245,147287,433180,7199,1355906,80150,69414,46064,0,2857427,421272,26900,6521,59914,59914,448465,11247,344880,150847,1683,1318,0
1,한화생명,2016.06,0,0,164247,314839,1030291,13024,0,695173,695173,1725465,1725465,140828,836001,0,3140451,5620176,-112,453536,632683,8468172,8830905,23699,496968,362732,0,0,0,0,0,1339980,81859,73863,54477,0,3037447,695285,0,0,71330,71330,459268,11085,239461,336037,0,0,0
2,한화생명,2017.06,0,0,203666,367103,238784,0,0,554750,554750,793535,793535,170826,725576,0,2717609,7580972,67766,727536,770901,12281683,12967162,40932,81029,685479,0,0,0,0,0,1487751,635983,514765,93769,0,4775857,486984,0,0,149901,151256,504779,9760,299437,1344803,0,0,0
3,한화생명,2018.06,0,0,235606,422129,-209391,0,-235807,303000,303000,93609,93609,139954,442955,0,2169576,7445382,55206,893027,671186,11767729,12206324,22245,26606,438594,97439,1115017,0,0,4167,1523883,714689,688006,57459,0,5587905,247794,0,-8747,106907,108055,601826,11879,0,76252,8563,0,0
4,한화생명,2019.06,0,0,231919,442462,0,0,0,88721,0,921323,921323,12194,100916,0,2156184,7737445,-1400,988012,645680,12537229,12631283,31776,38639,94054,0,0,0,0,0,1482700,764286,683487,73188,0,5764440,90122,0,0,121731,123253,475807,11100,1234440,220884,0,0,0
5,한화생명,2020.06,0,0,215766,377465,0,0,0,162726,0,1062193,1062193,55431,218157,0,2285804,8102274,4225,1015146,621055,13457529,13680290,17739,13137,222760,0,0,0,0,0,1491854,747414,714597,71600,0,6229515,158500,0,0,98064,98064,507108,29212,1521660,296218,0,0,0


### 통합 파일 저장

In [23]:
df_join_t.to_csv(f'{corp_nm}_{report_type_selected}_테스트.csv', encoding='utf-8-sig')

### 수정 마무리 및 저장

In [24]:
df = pd.read_csv(f'{corp_nm}_{report_type_selected}_테스트.csv')

In [25]:
df = df.rename(columns={'Unnamed: 0':'{}'.format('보고기간')}).copy()

In [26]:
# df['보고시점'] = df.iloc[:,0].str.replace('-01','')

In [27]:
df.to_csv(f'{corp_nm}_{report_type_selected}_선택기간{data_size}_통합파일.csv', encoding='utf-8-sig')